<center>

# **如何进行GLM 一阶分析**

</center>

***

本教程以 NARPS 数据为例，基于 [Nilearn 工具包](https://nilearn.github.io/stable/auto_examples/04_glm_first_level/index.html)，详细介绍一阶分析的基本流程。

本节介绍参数调制（Parametric Modulation）的分析策略，首先会介绍什么是Parametric Modulation，其次会以narps数据为例进行具体代码展示。

对于基于 GIfTI（.gii）格式的表层空间分析内容，本教程不作详细展开，感兴趣的读者可参考以下补充代码文件：

1. Step_4_Analysis/GLM/step4_surface_based_firstlevel.py
2. Step_4_Analysis/GLM/step4_surface_based_firstlevel_modulation.py

- **什么是Parametric Modulation**
- **Step 1: events文件的读取与预处理**
- **Step 2: 基于confounds文件的运动校正**
- **Step 3: 使用 FirstLevelModel 进行建模与估计**
- **Step 4: 设置条件对比（contrast）并导出结果**

# 什么是Parametric Modulation

在大多数 fMRI 实验分析中，我们使用**GLM**来估计每种实验条件（如视觉刺激、听觉刺激等）对大脑中每个体素的 BOLD 信号影响。这种方法会为每个条件生成一个参数估计值（*β*值），代表这个条件平均引发的 BOLD 响应。

但在一些实验中，我们不仅关心是否出现了某个刺激，更关心这个刺激在不同特征强度下是否会引起不同的大脑反应。这就是参数调制（Parametric Modulation）的用武之地。

假设你有一个任务，其中呈现一束光，每次光的亮度不同。如果你记录了每次光亮度的数值，你就可以检查：BOLD 信号是否随着亮度增加而增强？

这个“信号是否随着某种变量变化而变化”的过程，就是Parametric Modulation。

> https://andysbrainbook.readthedocs.io/en/latest/PM/PM_Overview.html

> Sabrina M. Tom et al. ,The Neural Basis of Loss Aversion in Decision-Making Under Risk.Science315,515-518(2007).DOI:10.1126/science.1134239


In [ ]:
# 导入所需的库
import os
import pandas as pd
import numpy as np
import os.path as op
import nibabel as nb
from nilearn.glm.first_level import FirstLevelModel
from nilearn.plotting import plot_stat_map
from nilearn.glm.first_level import make_first_level_design_matrix
# 用于计算不同条件之间的对比
from itertools import combinations 
from nilearn.plotting import plot_design_matrix
import matplotlib.pyplot as plt

In [ ]:
# 定义函数，根据头动参数（帧位移与六个方向的运动参数）识别头动过大的时间点并删除
def motion_controlled_event_timetable(event_table, fd_data, six_absolute_motion, TR, FD_thr, ab_motion_thr):
    # 检测超出帧位移（FD）阈值的时间点
    out_motion_detect = fd_data.to_numpy()[:] > FD_thr
    out_motion_index = np.where(out_motion_detect == True)
    
    # 检测超过绝对运动阈值的时间点
    six_motion_ex = np.where(np.sum((six_absolute_motion > ab_motion_thr) == True, 1) > 0)
    
    # 将运动时间点通过乘以 TR 转换为实际时间
    out_motion_time = np.array([])
    if len(out_motion_index[0]) > 0:
        out_motion_time = (out_motion_index[0][:] + 1) * TR
    if len(six_motion_ex[0]) > 0:
        six_motion_time = (six_motion_ex[0] + 1) * TR
        out_motion_time = np.concatenate((out_motion_time, six_motion_time), axis=0)
        out_motion_time = np.unique(out_motion_time)
    
    # 为事件表添加一个新列'time_end'，计算每个事件的结束时间
    # 使用lambda函数计算：结束时间 = 开始时间(onset) + 持续时间(duration)
    tmp_timetable = event_table.assign(time_end=lambda dataframe: dataframe['onset'] + dataframe['duration'])
    tmp_timetable = tmp_timetable.reset_index(drop=True)
    
    # 标记运动超过阈值的时间点
    block_time_judge = np.zeros(tmp_timetable.shape[0])
    block_time_in = np.zeros(tmp_timetable.shape[0])
    try:
        for n_time in range(tmp_timetable.shape[0]):
            for i in out_motion_time:
                time_judge_0 = (i <= tmp_timetable.loc[n_time, 'time_end'])
                block_time_judge[n_time] += time_judge_0
                time_judge_1 = (i <= tmp_timetable.loc[n_time, 'time_end']) * (i >= tmp_timetable.loc[n_time, 'onset'])
                block_time_in[n_time] += time_judge_1
            
        tmp_timetable = tmp_timetable.assign(
            time_delete=block_time_judge * TR,
            delete_time_inblock=block_time_in
        )
        tmp_timetable.loc[:, 'duration'] = tmp_timetable['duration'] - tmp_timetable['delete_time_inblock'] * TR
        
        # 调整起始时间 (onset) 并重新计算结束时间
        for n_time in range(tmp_timetable.shape[0]):
            if n_time != 0:
                tmp_timetable.loc[n_time, 'onset'] = tmp_timetable.loc[n_time, 'onset'] - tmp_timetable.loc[n_time, 'time_delete']
            tmp_timetable.loc[n_time, 'time_end'] = tmp_timetable.loc[n_time, 'onset'] + tmp_timetable.loc[n_time, 'duration']
    except Exception as e:
        print("Error in motion_controlled_event_timetable:", e)
        out_motion_time = False
        tmp_timetable = event_table
    return [tmp_timetable, out_motion_time]

In [ ]:
# 定义函数，删除对应运动时间点的 NIfTI 时间点
def correct_motion_for_niidata(motion_corrected_path, subname, run, task_file, nii_data, TR, out_motion_time):
    motion_corrected_subfolder = op.join(motion_corrected_path, subname)
    if not os.path.exists(motion_corrected_subfolder):
        os.makedirs(motion_corrected_subfolder)
    motion_corrected_nii = op.join(motion_corrected_subfolder, subname + task_file)
    niidata = nb.load(nii_data)
    timepoints_to_delete = ((out_motion_time / TR).astype('int64')) - 1
    motion_corrected_data = np.delete(niidata.get_fdata(), timepoints_to_delete, axis=3)
    motion_corrected_nii_data = nb.Nifti1Image(motion_corrected_data, header=niidata.header, affine=niidata.affine)
    motion_corrected_nii_data.header.set_data_dtype(np.int16)
    nb.save(motion_corrected_nii_data, motion_corrected_nii)
    return motion_corrected_nii

In [ ]:
# 设置数据根目录
rootdir = '/Volumes/ss/ds001734'
# 可以根据需要修改被试列表(sublist)和run列表(runs)
# 这里只使用Narps数据中的第一个被试，一共是4次run
sublist = ['001']  
runs = ['01', '02', '03', '04']
# task名称，可以根据实际数据修改
taskname = 'MGT'
# 根据你的数据修改TR时间
# 对于公开数据集，TR值通常可以在以下位置找到：
# 1. 原始论文的Methods部分
# 2. BIDS格式数据的task-*_bold.json文件中的"RepetitionTime"字段
TR = 1.0  
# 设置帧位移和绝对运动阈值，这是绝大多数研究中常用的阈值，你也可以根据实际情况进行调整
FD_thr = 0.2  
ab_motion_thr = 3  

# 设置输出路径
out_path = op.join(rootdir, 'derivatives', 'first_level_model_corrected_nii')
motion_corrected_path = op.join(rootdir, 'derivatives', 'motion_corrected_data_nii')

# 设置是否进行头动校正（删除头动过大的时间点）
# 设置为 False 可避免删除由于头动造成的运动伪影较大的时间帧
do_motion_exclusion = False  


在实际的 GLM 分析中，我们通常不会只处理一个被试，因此使用循环结构可以大幅提高分析效率。

不过，如果你希望深入理解循环代码中每一行的含义，以及各个变量的具体内容，可以在每行代码下方添加 `print()` 语句，或者单独创建一个代码块，打印变量的输出结果来进行查看。


In [ ]:
# 以`subeventdir` 为例
# 取消下面代码的注释，运行即可查看`subeventdir`变量的输出结果是什么
# subeventdir

# 或者使用print语句查看，取消注释运行查看输出结果即可
# print(f"events文件的路径：{subeventdir}")

关于nilearn中 'FirstLevelModel' 各参数的介绍，可以参考：

> https://nilearn.github.io/dev/auto_examples/04_glm_first_level/plot_first_level_details.html

In [ ]:
# 主要代码
for sub in sublist:
    subname = 'sub-' + sub
    subeventdir = op.join(rootdir, subname, 'func')
    subimagedir = op.join(rootdir, 'derivatives', 'fmriprep', subname, 'func')
    for run in runs:
        sub_event_file = op.join(subeventdir, f'{subname}_task-{taskname}_run-{run}_events.tsv')
        sub_niidata_file = op.join(subimagedir, f'{subname}_task-{taskname}_run-{run}_bold_space-MNI152NLin2009cAsym_preproc.nii.gz')
        sub_motioninfo_file = op.join(subimagedir, f'{subname}_task-{taskname}_run-{run}_bold_confounds.tsv')

        # Check if all necessary files exist
        if not (os.path.exists(sub_event_file) and os.path.exists(sub_niidata_file) and os.path.exists(sub_motioninfo_file)):
            print(f"Missing files for {subname}, run {run}")
            continue

        # Load event data
        event_data = pd.read_csv(sub_event_file, sep='\t')
        # Rename 'participant_response' to 'trial_type', here we use the condition based coding for each trial
        # make trial_type based on whether the participant_response equal to 'NoResp'
        event_data['trial_type'] = event_data['participant_response'].apply(
            lambda x: 'NoResp' if x == 'NoResp' else 'Resp'
        )

        # make sure that gain as modulation effect
        if 'gain' not in event_data.columns:
            print(f"'gain' column not found in {sub_event_file}")
            continue
        else:
            event_data.rename(columns={'gain': 'modulation'}, inplace=True)
        # Load motion parameters data
        confounds = pd.read_csv(sub_motioninfo_file, sep='\t')
        # Handle missing values
        confounds = confounds.fillna(0)

        # Get framewise displacement (FD) data
        if 'FramewiseDisplacement' in confounds.columns:
            fd = confounds[['FramewiseDisplacement']]
        elif 'framewise_displacement' in confounds.columns:
            fd = confounds[['framewise_displacement']]
        else:
            print(f"FD column not found in {sub_motioninfo_file}")
            continue

        # Get six motion parameters
        motion_params = confounds[['X', 'Y', 'Z', 'RotX', 'RotY', 'RotZ']]

        # Motion correction
        if do_motion_exclusion:
            [event_data_corrected, out_motion_time] = motion_controlled_event_timetable(event_data, fd, motion_params, TR, FD_thr, ab_motion_thr)
            # Save corrected event data
            out_sub_path = op.join(out_path, subname)
            if not os.path.exists(out_sub_path):
                os.makedirs(out_sub_path)
            event_out_file = op.join(out_sub_path, f'{subname}_task-{taskname}_run-{run}_events_corrected.tsv')
            event_data_corrected.to_csv(event_out_file, sep='\t', index=False)
            # Correct NIfTI data
            if not isinstance(out_motion_time, bool):
                task_file = f'_task-{taskname}_run-{run}_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'
                corrected_nii_file = correct_motion_for_niidata(motion_corrected_path, subname, run, task_file, sub_niidata_file, TR, out_motion_time)
                fmri_img = nb.load(corrected_nii_file)
                # Adjust motion parameters
                timepoints_to_delete = ((out_motion_time / TR).astype('int64')) - 1
                motion_params_corrected = motion_params.drop(motion_params.index[timepoints_to_delete]).reset_index(drop=True)
            else:
                fmri_img = nb.load(sub_niidata_file)
                motion_params_corrected = motion_params
                event_data_corrected = event_data
        else:
            event_data_corrected = event_data
            fmri_img = nb.load(sub_niidata_file)
            motion_params_corrected = motion_params
            
        frame_times = (
            np.arange(fmri_img.shape[3]) * TR
        )  # here are the corresponding frame times
        
        unmodulated_matrix = event_data_corrected[['onset', 'duration', 'trial_type']]
        modulated_matrix = event_data_corrected[['onset', 'duration', 'trial_type','modulation']]
        
        GLM_matrix_unmodulated = make_first_level_design_matrix(
            frame_times,
            unmodulated_matrix,
            drift_model="polynomial",
            drift_order=3,
            hrf_model="spm + derivative",
        )
        
        GLM_matrix_modulated = make_first_level_design_matrix(
            frame_times,
            modulated_matrix,
            drift_model="polynomial",
            drift_order=3,
            hrf_model="spm + derivative",
        )
        # Let's compare two design matrix
        fig, (ax1, ax2) = plt.subplots(
            figsize=(10, 6), nrows=1, ncols=2, constrained_layout=True
        )

        plot_design_matrix(GLM_matrix_unmodulated, axes=ax1,rescale=False)
        ax1.set_title("Event design matrix", fontsize=12)
        plot_design_matrix(GLM_matrix_modulated, axes=ax2,rescale=False)
        ax2.set_title("Modulated Event design matrix", fontsize=12)
        plt.show()

        # Perform first-level GLM analysis
        fmri_glm = FirstLevelModel(
            t_r=TR,
            noise_model='ar1',
            hrf_model='spm + derivative',
            drift_model='polynomial',# the desired drift model for the design matrices
            drift_order = 3,  # Adjust the drift order as needed
            high_pass=1./128,  # Adjust the high-pass filter as needed
            signal_scaling=False,  # Whether to scale the signal
            minimize_memory=False
        )
        # we use the GLM_matrix_modulated
        fmri_glm = fmri_glm.fit(
            fmri_img,
            events=modulated_matrix,
            confounds=motion_params_corrected
        )

        # Define contrasts
        # Assuming 'trial_type' column contains condition names
        conditions = event_data_corrected['trial_type'].unique()
        design_matrix = fmri_glm.design_matrices_[0]
        # Create contrasts for each condition
        contrasts = {}
        for cond in conditions:
            # The columns corresponding to the condition
            cond_vector = np.array([1 if c == cond else 0 for c in design_matrix.columns])
            contrasts[cond] = cond_vector
        # Compute contrasts for each condition vs baseline
        out_sub_path = op.join(out_path, subname)
        stats_results_path = op.join(out_sub_path, 'stats_results', f'run-{run}')
        if not os.path.exists(stats_results_path):
            os.makedirs(stats_results_path)
        for contrast_id, contrast_val in contrasts.items():
            z_map = fmri_glm.compute_contrast(contrast_val, output_type='z_score')
            z_map_file = op.join(stats_results_path, f'{subname}_task-{taskname}_run-{run}_{contrast_id}_zmap.nii.gz')
            z_map.to_filename(z_map_file)
            print(f"Contrast {contrast_id} for {subname}, run {run} completed and saved to {z_map_file}")
            # Optionally, plot the contrast map
            plot_stat_map(z_map, title=f'{subname} {contrast_id}', display_mode='ortho', threshold=1.0)# unthreshold stats map
